<a href="https://colab.research.google.com/github/hargurjeet/Adhoc-Activities/blob/main/Copy_of_rag_with_oops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q langchain langchain-community langchain-chroma langchain-huggingface pypdf torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain_chroma import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [19]:
class RAGPipeline():
  def __init__(self, file_path, query):
    self.file_path = file_path
    self.chunk_size=500
    self.chunk_overlap=50
    self.model_name = "sentence-transformers/all-mpnet-base-v2"
    self.model_kwargs = {'device': 'cpu'}
    self.encode_kwargs = {'normalize_embeddings': False}
    self.model_id = "gpt2"
    self.system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
    )
    self.query = query

  def chunk_load(self):
    loader = PyPDFLoader(self.file_path)
    raw_docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=self.chunk_size,
    chunk_overlap=self.chunk_overlap,
    length_function=len,
    is_separator_regex=False,
    )

    return text_splitter.split_documents(raw_docs)

  def embedder(self):
    return HuggingFaceEmbeddings(model_name=self.model_name,model_kwargs=self.model_kwargs,encode_kwargs=self.encode_kwargs)

  def llm_new(self):
    tokenizer = AutoTokenizer.from_pretrained(self.model_id)
    model = AutoModelForCausalLM.from_pretrained(self.model_id)
    pipe = pipeline(
        "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
    )
    return HuggingFacePipeline(pipeline=pipe)

  def vector_store_df(self):
    chunks = self.chunk_load()
    hf_embedding = self.embedder()
    return Chroma.from_documents(chunks, hf_embedding)

  def retriever(self):
    db = self.vector_store_df()
    return db.as_retriever()

  def retertival_qa(self, query):
    prompt = ChatPromptTemplate.from_messages(
    [
        ("system", self.system_prompt),
        ("human", "{input}"),
    ]
  )
    question_answer_chain = create_stuff_documents_chain(self.llm_new(), prompt)
    chain = create_retrieval_chain(self.retriever(), question_answer_chain)

    response = chain.invoke({"input": query})
    return response['answer']






In [20]:
if __name__ == "__main__":

  file_path = "/content/Diet Plan PDF_250206_114727.pdf"
  query = "Can you tell me about diet plan"
  rag_pipeline = RAGPipeline(file_path, query)

In [21]:
rag_pipeline.retertival_qa(query)

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"System: Use the given context to answer the question. If you don't know the answer, say you don't know. Use three sentence maximum and keep the answer concise. Context: - Limit oil intake to 3 tsp per day - Make sure\nsalad is taken 10-15 minutes before lunch and dinner. - Consume a minimum 3 lt of\nwater per day. - Minor meals are compulsory and try to avoid extra carbs in minor\nmeals. Prefer fruits/protein (chana,sprouts,egg whites) or veggies as options.\nQuantity Guidelines\n \nKatori\n150ml / 5.07oz\n \nSmall Bowl\n150ml / 5.07oz\n \nBowl\n350ml / 11.83oz\n \nCup\n250ml / 8.5oz\n \nTea Cup\n180ml / 6.08oz\n \nGlass\n200ml / 6.76oz\n \nTablespoon\n15ml / 0.51oz\n \nTeaspoon\n\n- Limit oil intake to 3 tsp per day - Make sure\nsalad is taken 10-15 minutes before lunch and dinner. - Consume a minimum 3 lt of\nwater per day. - Minor meals are compulsory and try to avoid extra carbs in minor\nmeals. Prefer fruits/protein (chana,sprouts,egg whites) or veggies as options.\nQuantity Guid